In [1]:
!pip install ../pyldpc

Processing /media/charles/SSD/Fuzzy Extractor and Fingerprint/pyldpc
  Created wheel for pyldpc: filename=pyldpc-1.1.5-py3-none-any.whl size=16268 sha256=3330992309b3089d367f6985af9efee0308d7baa4628773bd13f9395e478ea23
  Stored in directory: /tmp/pip-ephem-wheel-cache-hxrfqtw8/wheels/ea/3c/ee/110d4c98c20ce323971bbebafd1ba3ab27472b8b953d2d8219
Successfully built pyldpc
  Attempting uninstall: pyldpc
    Found existing installation: pyldpc 1.1.5
    Uninstalling pyldpc-1.1.5:
      Successfully uninstalled pyldpc-1.1.5


In [1]:
# Author: Hicham Janati (hicham.janati@inria.fr)
#
# License: BSD (3-clause)


import numpy as np

from matplotlib import pyplot as plt
from PIL import Image

from time import time
import sys

sys.path.append("../")
from pyldpc import make_ldpc, ldpc_images
from pyldpc.utils_img import gray2bin, rgb2bin
# from python_fuzzy_extractor.fuzzy_extractor import FuzzyExtractor
from pyldpc.LDPC_FE_Util import LDPC_FE_Util


In [2]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message
n = 128 # code length
d_v = 3 #ones per row
d_c = 16
snr = 100
# First we create a small LDPC code i.e a pair of decoding and 
# coding matrices H and G. H is a regular parity-check matrix with d_v ones per row and d_c ones per column
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1] # message length
print(k)

106


In [4]:
# v = np.random.randint(2, size=k)
# print(v,'secret')
# y = encode(G, v)# y is polr code
# print('encode',len(y))
# print('encode',y)
# y = y+np.random.rand(n)# here add noise
# # print('encode',y)
# d = decode(H,y,snr)
# # print(d)
# x = get_message(G, d)
# print(x)
# assert abs(x - v).sum() == 0

In [5]:
# # paramters
# # enroll
# total_length = 16 # length of input of fuzzy extractor
# #The number of bits that can be flipped in the source value and 
# # still produce the same key with probability (1 - rep_err).
# error_precision = 8
# top_score = 16# factor of 16
# blocks = int(top_score / total_length)
# using_max_entropy_or_random_perm = True
# perm_feat = False# use perm for each user?

In [6]:
# import sys
# sys.path.append("../")
# from python_fuzzy_extractor.fuzzy_extractor import FuzzyExtractor

# extractor = FuzzyExtractor(total_length, error_precision)

In [7]:
# key, helper = extractor.generate(s)

In [8]:
# from os import urandom
# key = np.fromstring(urandom(16), dtype=np.uint8)
# print(key)
# y = encode(G, key)# y is polr code
# print('encode',y)

In [9]:
def genW(n,q,m):# m, hashcode length, q sub-dimentsion n data dimension
    return np.random.rand(m,n,q)

def IoMhashing(x,ws):
    para = np.shape(ws)
    hashcode = np.zeros(para[0])
    for i in range(para[0]):
        hashcode[i] = np.argmax(np.matmul(x,ws[i]))
    hashcode = hashcode.astype(int)
    return hashcode

def bin_array(num, m):
    """Convert a positive integer num into an m-bit bit vector"""
    return np.array(list(np.binary_repr(num).zfill(m))).astype(np.int8)
def binary_to_decimal(binary):
    i,integer = 0,0
    size = len(binary)
    while i < len(binary):
        integer += int(binary[size - 1 - i])*pow(2,i)
        i+=1
    return integer
binary_to_decimal([0,1,0])


n = 512
biometric = np.random.rand(n)
biometric2 = np.random.rand(n)
biometric3 = biometric+np.random.rand(n)*0.01 # a mated sample

W = genW(512,8,64)
hashcode = IoMhashing(biometric,W)
hashcode2 = IoMhashing(biometric2,W)
hashcode3 = IoMhashing(biometric3,W)
print(np.linalg.norm(hashcode - hashcode2, axis=-1)/(np.linalg.norm(hashcode)+np.linalg.norm(hashcode2)))
print(np.linalg.norm(hashcode - hashcode3, axis=-1)/(np.linalg.norm(hashcode)+np.linalg.norm(hashcode3)),'mated')
print(hashcode)

0.3559951722117125
0.0 mated
[6 6 1 4 5 4 1 3 3 6 0 7 7 1 3 6 6 6 0 2 5 3 5 4 0 6 0 4 6 3 5 0 4 2 3 3 5
 2 3 2 4 6 2 1 7 4 7 1 3 7 3 0 5 3 0 3 3 3 4 6 4 7 7 2]


In [10]:
hashcode = np.array([LDPC_FE_Util.bin_array(k,8) for k in hashcode]).reshape(-1)
hashcode2 = np.array([bin_array(k,8) for k in hashcode2]).reshape(-1)
hashcode3 = np.array([bin_array(k,8) for k in hashcode3]).reshape(-1)
# hashcode[hashcode==0] = -1
# hashcode2[hashcode2==0] = -1
# hashcode3[hashcode3==0] = -1


TypeError: bin_array() missing 1 required positional argument: 'm'

In [ ]:
lDPC_FE_Util = LDPC_FE_Util(512,3,4)# message 418 bits, encode into 512 bits
# d_c must divide n for a regular LDPC matrix H
# fuzzy vault 18 ints as key, 16bit + 2 bit error
# 18*8=144 bits key length LDPC_FE_Util(119,3,16)
keys,keys_int = lDPC_FE_Util.genKey()
# print(np.shape(keys))
# print(np.shape(keys_int))
ec,ec_int = lDPC_FE_Util.encode(keys)
# print(len(ec))
# print(len(ec_int))
# print(ec_int)
decode_key, de_key_int = lDPC_FE_Util.decode(ec)
# print(decode_key)
assert abs(keys - decode_key).sum() == 0

In [ ]:
ciphers1 = np.bitwise_xor(hashcode, ec)
ciphers1

In [ ]:
ciphers2 = np.bitwise_xor(hashcode2, ec_int)
print(ciphers2)
decode_key, de_key_int = lDPC_FE_Util.decode(ciphers2)
assert abs(keys - decode_key).sum() == 0

In [ ]:
ciphers3 = np.bitwise_xor(hashcode3, ec_int)
decode_key, de_key_int = lDPC_FE_Util.decode(ciphers3)
assert abs(keys - decode_key).sum() == 0

In [ ]:
# key = np.fromstring(urandom(16), dtype=np.uint8)
# key_arr = np.array([bin_array(k,8) for k in key])
# # print(key_arr)
# # print(key_arr.reshape(-1))
# bin_key = key_arr.reshape(-1)
# key_pad = np.concatenate((bin_key, np.zeros(2, dtype=np.uint8)))
# y = encode(G, key_pad)# y is polr code
# # print('encode',y)# encode into 512 bits
# y[y==-1] =0
# ciphers = np.bitwise_xor(hashcode, y)
# print(ciphers)

In [ ]:
plains = np.bitwise_xor(hashcode2, ciphers)
# print(plains)
# y = y+np.random.rand(n)# here add noise
# print('encode',y)
snr=100
d = decode(H,plains,snr)
# print(d)
x = get_message(G, d)
# print(x)
assert abs(x - key_pad).sum() == 0

In [ ]:
plains = np.bitwise_xor(hashcode3, ciphers)
# print(plains)
# y = y+np.random.rand(n)# here add noise
# print('encode',y)
snr=100
d = decode(H,plains,snr)
# print(d)
x = get_message(G, d)
# print(x)
assert abs(x - key_pad).sum() == 0

In [37]:
from os import urandom
key = np.fromstring(urandom(18), dtype=np.uint8)
print(key)

[135  92  22 198 101  60 101  83 228 115  34  84  20 240 153 101 161 122]


<ipython-input-37-9fecab460687>:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  key = np.fromstring(urandom(18), dtype=np.uint8)


In [ ]:
from fastpbkdf2 import pbkdf2_hmac
np.fromstring(pbkdf2_hmac('sha256', [1,2,3], [1,2,3], 1, 2), dtype=np.uint8)

In [ ]:
# print(len(ciphers)/8)
int_arr = [binary_to_decimal(ciphers[i*8:(i+1)*8]) for i in range(int(len(ciphers)/8))]
print(len(int_arr))